In [1]:
import os
import sys
from tqdm import tqdm 
from bs4 import BeautifulSoup 
import time
import requests

In [ ]:
# read cookies from config.json
import json
with open('../config.json', 'r') as f:
    config = json.load(f)
cookies = config['DCM']['cookies']
cookies

In [47]:
# set headers
headers = { 'Cookie': cookies, 
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0', 
           'Referer': 'https://download.geoservice.dlr.de'}

In [60]:
min_lon = 100
max_lon = 120
min_lat = 30
max_lat = 40

In [71]:
with tqdm(total=(max_lat-min_lat)*(max_lon-min_lon)) as pbar:
    for lat in range(min_lat, max_lat):
        for lon in range(min_lon, max_lon):
            round_lon = lon//10*10
            url = f'https://download.geoservice.dlr.de/TDM30_DCM/files/TDM1_DCM__10_N{lat}/TDM1_DCM__10_N{lat}E{round_lon}/TDM1_DCM__10_N{lat}E{lon}_FIRST1622_V01_C/'
            res = requests.get(url, headers=headers)
            soup = BeautifulSoup(res.text, 'html.parser')
            a_tags = soup.find_all('a')
            tif_hrefs = [tag['href'] for tag in a_tags if tag['href'].endswith('.tif')]
            # download all tif files
            for tif_href in tif_hrefs:
                # sleep 1s
                time.sleep(1)
                tif_url = url + tif_href
                local_filename = os.path.join('J:\DCM', tif_href)
                if os.path.exists(local_filename):
                    pass
                else:
                    # 发送HTTP GET请求到指定的URL
                    with requests.get(tif_url, stream=True,headers=headers) as r:
                        r.raise_for_status()
                        # 将响应内容写入本地文件
                        with open(local_filename, 'wb') as f:
                                for chunk in r.iter_content(chunk_size=8192):
                                    f.write(chunk)
                pbar.update(1)
    pbar.close()
            

Exception ignored in: <function tqdm.__del__ at 0x0000016C5238BD00>
Traceback (most recent call last):
  File "c:\Users\Administrator\.conda\envs\ox\lib\site-packages\tqdm\std.py", line 1162, in __del__
    self.close()
  File "c:\Users\Administrator\.conda\envs\ox\lib\site-packages\tqdm\std.py", line 1291, in close
    if self.last_print_t < self.start_t + self.delay:
AttributeError: 'tqdm' object has no attribute 'last_print_t'
496it [54:02,  6.54s/it]
